In [166]:
import cv2
import numpy as np
from sklearn.metrics import mean_squared_error
import time

In [167]:
img1 = np.random.randint(256, size=(720, 1280, 3), dtype=np.uint8)
img2 = np.random.randint(256, size=(720, 1280, 3), dtype=np.uint8)
img1, img2 = img1.astype(np.float), img2.astype(np.float)

In [171]:
def mse(arr1, arr2):
    # Mean Squre Error between two images
    assert isinstance(arr1, np.ndarray) and isinstance(arr2, np.ndarray)
    assert arr1.shape == arr2.shape, "{}, {}".format(arr1.shape, arr2.shape)

#     return np.mean((arr1.astype(np.float) - arr2.astype(np.float))**2)

#     return mean_squared_error(arr1.astype(np.float).reshape(-1), arr2.astype(np.float).reshape(-1))

# row-by-row
    sum_squared_error = 0.
    for x in range(arr1.shape[0]):
        sum_squared_error += np.sum((arr1[x,:,:] - arr2[x,:,:])**2)
  

#     sum_squared_error = 0.
#     for x in range(arr1.shape[0]):
#         for y in range(0, arr1.shape[1], 320): # chunk of 8 in y
#             sum_squared_error += np.sum((arr1[x,y:y+320,:] - arr2[x,y:y+320,:])**2)

#     sum_squared_error = 0.
#     for x in range(arr1.shape[0]):
#         for z in range(arr1.shape[2]): # by channel
#             sum_squared_error += np.sum((arr1[x,:,z] - arr2[x,:,z])**2)

                        
    return sum_squared_error / arr1.size

In [172]:
tic = time.time()
repeat = 100
for _ in range(repeat):
    _ = mse(img1, img2)

elapsed = time.time() - tic
print "Total {:1f}s, {:.3f} ms / call".format(elapsed, 1000*elapsed/repeat)

Total 1.367418s, 13.674 ms / call


In [42]:
# GT= 10913
# original: 27 ms / call
# sklearn mean_squared_error: 31 ms / call
# row by  row: 15 ms / call

In [168]:
# TODO: trying multithreading using https://docs.python.org/2.7/library/multiprocessing.html#module-multiprocessing.dummy
import itertools

def sum_squared_error((arr1, arr2, n, m)):
#     print "get", n,m
#     print arr1.shape, arr2.shape
    rv = 0
    for x in range(n, arr1.shape[0], m):
        rv += np.sum((arr1[x,:,:] - arr2[x,:,:])**2)
    return rv

def mse_parallel(arr1, arr2, pool):
    total = 0.

    it = pool.imap(sum_squared_error, map(lambda n: (arr1, arr2, n, 4), range(4)))
    rv = np.sum(list(it)) / np.prod(arr1.size)
#     print rv
    return rv
    
#     for x in range(arr1.shape[0]):
#         pool.apply_async(sum_squared_error, (arr1[x,:,:], arr2[x,:,:]))

In [169]:
from multiprocessing.dummy import Pool

In [170]:
pool = Pool(processes=4)
tic = time.time()
repeat = 50
for _ in range(repeat):
    _ = mse_parallel(img1, img2, pool)

elapsed = time.time() - tic
print "Total {:1f}s, {:.3f} ms / call".format(elapsed, 1000*elapsed/repeat)

Total 2.072423s, 41.448 ms / call


In [103]:
zip(img1, img2)[0]

(array([[ 53.,  23.,  79.],
        [ 81., 227.,  88.],
        [244., 112., 101.],
        ...,
        [178., 237., 120.],
        [ 19., 238., 221.],
        [ 90., 114., 237.]]), array([[254.,  74.,  48.],
        [139., 206., 153.],
        [156.,  19., 211.],
        ...,
        [ 36., 231., 138.],
        [179., 134., 195.],
        [110., 112.,  65.]]))